# Parallel code for A364656: Number of strict interval closure operators on a set of n elements. 

All the values do not count closure system {{},[n]} since it forms a reducible context; so, we need to add 1 to the obtained results.

In [1]:
%load_ext Cython


In [2]:
%%cython --annotate
from math import log 

cpdef ProcessRT(t,int i,int l,int nn1):
    """This function recursively extends an input tuple of integers, which represents 
    a set of sets (more precisily, a formal context), by adding to it an integer i (a new set).
    The length of a terminal tuple is constrained by l, while nn1 should be passed as 2**n-1. 
    The function checks whether the new tuple forms a unique closure system and returns the dictionary
    of the first elements of processed tuples as keys and values 1 for clousre system with
    interval closure propery and 0 otherwise."""
    
    count={}
    t=t[:]
    cdef int P
    cdef int lent=len(t)

    
    #checking whether the context is reducible
    for j in range(lent-1,-1,-1):
        P=i
        #print("t[j],i",t[j],i,t[j]&i,)
        tji=t[j]&i
        if tji in t[0:j]:
            return count
            
        if tji==t[j]:
            #print("t[j],i",t[j],i,t[j]&i,)
            for h in range(j+1,lent):
                if t[j]&t[h]==t[j]:
                    P=P&t[h]
                    if P==t[j]:                        
                        return count
    t.append(i)

 
    if Check(t,nn1):
        count[(lent+1,t[0])]=1
    #else:
    #    count[(lent+1,t[0])]=0

    #recursive call
    cdef unsigned long long r
    #print(l)
    if l!=0:
        r=0
        for k in range(i+1,nn1):
            #print("Process(t,k,l-1,tuples,nn1)")
            #print("t",t,"k",k)
            ucount=ProcessRT(t,k,l-1,nn1)
            for key in ucount:
                if key in count:
                    count[key]+=ucount[key]
                else:
                    count[key]=ucount[key]
            #if r<0: print("r",r)
        return count
    else: 
        #print(tuples)
        #tuples.append(tuple(t))
       
        
        print("Branch process",t[0])
        return count

    
def isClosed(t,g):
    """Checks whether g is closed in t."""
    cont=[]
    for h in t:
        if g&h==g:
            cont.append(h)
    if len(cont)>0:
        res=cont[0]
        for h in cont:
            res=h&res
        return res==g
    else:
        return g==0
    
def Closure(t,g):
    """Returns closure of g in t."""
    res=~0
    for h in t:
        if g&h==g:
            res=h&res
    return res
    
    
def Check(t,nn1):
    """Checks whether {x,y}'' is subset of S is fulfilled for t."""
    t=t+[nn1]
    #print(t)
    n=int(log(nn1+1,2))
    members=[]
    members.extend([2**i for i in range(n)])
    powerset=range(nn1)
    lm=len(members)
    pairs=[members[i]|members[j] for i in range(lm) for j in range(i+1,lm)]
    closures={}
    for p in pairs:
        closures[p]=Closure(t,p)
    for s in powerset:
        mode=True
#         for i in range(lm):
#             for j in range(i+1,lm):
#                 p=members[i]|members[j]
        for p in pairs:   
            #cl=Closure(t,p)
            if p&s==p:
                if closures[p]&s!=closures[p]:
                        #print("False")
                        #print(p,cl,s)
                    mode=False
                    break
            #if mode==False: break           
                        
        if mode:
            if s not in members and Closure(t,s)!=s:
                #print("False")
                #print(t,s)
                #print(Closure(t,s))
                return False
                        
            
    return True


In [3]:
def merge(r):
    """This is a value collector for the resulting list of dictionaries
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged:
                merged[key]+=dic[key]
            else:
                merged[key]=dic[key]
    return merged



In [4]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=2

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res2 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res2.get())# print the list of resulting dictionaries
    print(sum(merge(res2.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 2, 3), ([0], 2, 2, 3), ([1], 2, 2, 3)]
[{(2, 0): 1}, {(2, 0): 1}, {(2, 1): 1}]
3
CPU times: user 13.5 ms, sys: 32.2 ms, total: 45.6 ms
Wall time: 48.4 ms


In [5]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=3

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res3 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res3.get())# print the list of resulting dictionaries
    print(sum(merge(res3.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 6, 7), ([0], 2, 6, 7), ([0], 3, 6, 7), ([0], 4, 6, 7), ([0], 5, 6, 7), ([0], 6, 6, 7), ([1], 2, 6, 7), ([1], 3, 6, 7), ([1], 4, 6, 7), ([1], 5, 6, 7), ([1], 6, 6, 7), ([2], 3, 6, 7), ([2], 4, 6, 7), ([2], 5, 6, 7), ([2], 6, 6, 7), ([3], 4, 6, 7), ([3], 5, 6, 7), ([3], 6, 6, 7), ([4], 5, 6, 7), ([4], 6, 6, 7), ([5], 6, 6, 7)]
[{(2, 0): 1, (3, 0): 2}, {(2, 0): 1, (3, 0): 2}, {(2, 0): 1, (3, 0): 2}, {(2, 0): 1, (3, 0): 2}, {(2, 0): 1, (3, 0): 1}, {(2, 0): 1}, {(2, 1): 1, (3, 1): 4, (4, 1): 4}, {(3, 1): 2, (4, 1): 1}, {(2, 1): 1, (3, 1): 2}, {(3, 1): 1}, {(2, 1): 1}, {(3, 2): 2, (4, 2): 1}, {(2, 2): 1, (3, 2): 2}, {(2, 2): 1, (3, 2): 1}, {}, {(2, 3): 1, (3, 3): 2}, {(3, 3): 1}, {}, {}, {}, {}]
44
CPU times: user 8.71 ms, sys: 36.4 ms, total: 45.2 ms
Wall time: 72.4 ms


In [21]:
res3.get()

[{(2, (0, 1)): 1, (3, (0, 1, 3)): 1, (3, (0, 1, 5)): 1},
 {(2, (0, 2)): 1, (3, (0, 2, 3)): 1, (3, (0, 2, 6)): 1},
 {(2, (0, 3)): 1, (3, (0, 3, 5)): 1, (3, (0, 3, 6)): 1},
 {(2, (0, 4)): 1, (3, (0, 4, 5)): 1, (3, (0, 4, 6)): 1},
 {(2, (0, 5)): 1, (3, (0, 5, 6)): 1},
 {(2, (0, 6)): 1},
 {(2, (1, 2)): 1,
  (3, (1, 2, 3)): 1,
  (4, (1, 2, 3, 4)): 1,
  (3, (1, 2, 4)): 1,
  (4, (1, 2, 4, 5)): 1,
  (4, (1, 2, 4, 6)): 1,
  (3, (1, 2, 5)): 1,
  (4, (1, 2, 5, 6)): 1,
  (3, (1, 2, 6)): 1},
 {(2, (1, 3)): 0, (3, (1, 3, 4)): 1, (4, (1, 3, 4, 6)): 1, (3, (1, 3, 6)): 1},
 {(2, (1, 4)): 1, (3, (1, 4, 5)): 1, (3, (1, 4, 6)): 1},
 {(2, (1, 5)): 0, (3, (1, 5, 6)): 1},
 {(2, (1, 6)): 1},
 {(2, (2, 3)): 0, (3, (2, 3, 4)): 1, (4, (2, 3, 4, 5)): 1, (3, (2, 3, 5)): 1},
 {(2, (2, 4)): 1, (3, (2, 4, 5)): 1, (3, (2, 4, 6)): 1},
 {(2, (2, 5)): 1, (3, (2, 5, 6)): 1},
 {(2, (2, 6)): 0},
 {(2, (3, 4)): 1, (3, (3, 4, 5)): 1, (3, (3, 4, 6)): 1},
 {(2, (3, 5)): 0, (3, (3, 5, 6)): 1},
 {(2, (3, 6)): 0},
 {(2, (4, 5)): 0

In [13]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=4

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res4 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res4.get())# print the list of resulting dictionaries
    print(sum(merge(res4.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 14, 15), ([0], 2, 14, 15), ([0], 3, 14, 15), ([0], 4, 14, 15), ([0], 5, 14, 15), ([0], 6, 14, 15), ([0], 7, 14, 15), ([0], 8, 14, 15), ([0], 9, 14, 15), ([0], 10, 14, 15), ([0], 11, 14, 15), ([0], 12, 14, 15), ([0], 13, 14, 15), ([0], 14, 14, 15), ([1], 2, 14, 15), ([1], 3, 14, 15), ([1], 4, 14, 15), ([1], 5, 14, 15), ([1], 6, 14, 15), ([1], 7, 14, 15), ([1], 8, 14, 15), ([1], 9, 14, 15), ([1], 10, 14, 15), ([1], 11, 14, 15), ([1], 12, 14, 15), ([1], 13, 14, 15), ([1], 14, 14, 15), ([2], 3, 14, 15), ([2], 4, 14, 15), ([2], 5, 14, 15), ([2], 6, 14, 15), ([2], 7, 14, 15), ([2], 8, 14, 15), ([2], 9, 14, 15), ([2], 10, 14, 15), ([2], 11, 14, 15), ([2], 12, 14, 15), ([2], 13, 14, 15), ([2], 14, 14, 15), ([3], 4, 14, 15), ([3], 5, 14, 15), ([3], 6, 14, 15), ([3], 7, 14, 15), ([3], 8, 14, 15), ([3], 9, 14, 15), ([3], 10, 14, 15), ([3], 11, 14, 15), ([3], 12, 14, 15), ([3], 13, 14, 15), ([3], 14, 14, 15), ([4], 5, 14, 15), ([4], 6, 14, 15), ([4], 7, 14, 15), ([4], 8, 14, 15), ([4], 9

In [14]:
res4.get()

[{(2, 0): 1, (3, 0): 6, (4, 0): 9},
 {(2, 0): 1, (3, 0): 6, (4, 0): 9},
 {(2, 0): 1, (3, 0): 8, (4, 0): 18, (5, 0): 10},
 {(2, 0): 1, (3, 0): 6, (4, 0): 9},
 {(2, 0): 1, (3, 0): 7, (4, 0): 14, (5, 0): 6},
 {(2, 0): 1, (3, 0): 6, (4, 0): 10, (5, 0): 2},
 {(2, 0): 1, (3, 0): 6, (4, 0): 9},
 {(2, 0): 1, (3, 0): 6, (4, 0): 9},
 {(2, 0): 1, (3, 0): 5, (4, 0): 9, (5, 0): 5},
 {(2, 0): 1, (3, 0): 4, (4, 0): 5, (5, 0): 1},
 {(2, 0): 1, (3, 0): 3, (4, 0): 3},
 {(2, 0): 1, (3, 0): 2},
 {(2, 0): 1, (3, 0): 1},
 {(2, 0): 1},
 {(2, 1): 1, (3, 1): 12, (4, 1): 54, (5, 1): 107, (6, 1): 79, (7, 1): 12},
 {(3, 1): 6, (4, 1): 27, (5, 1): 43, (6, 1): 29, (7, 1): 5},
 {(2, 1): 1, (3, 1): 10, (4, 1): 36, (5, 1): 52, (6, 1): 20, (7, 1): 1},
 {(3, 1): 5, (4, 1): 20, (5, 1): 25, (6, 1): 11},
 {(2, 1): 1, (3, 1): 8, (4, 1): 25, (5, 1): 31, (6, 1): 5},
 {(3, 1): 4, (4, 1): 14, (5, 1): 11},
 {(2, 1): 1, (3, 1): 6, (4, 1): 9},
 {(3, 1): 3, (4, 1): 9, (5, 1): 5},
 {(2, 1): 1, (3, 1): 4, (4, 1): 5, (5, 1): 1},
 {(3,

In [15]:
len(merge(res4.get()).values())

39

In [7]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=5

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res5 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res5.get())# print the list of resulting dictionaries
    print(sum(merge(res5.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 30, 31), ([0], 2, 30, 31), ([0], 3, 30, 31), ([0], 4, 30, 31), ([0], 5, 30, 31), ([0], 6, 30, 31), ([0], 7, 30, 31), ([0], 8, 30, 31), ([0], 9, 30, 31), ([0], 10, 30, 31), ([0], 11, 30, 31), ([0], 12, 30, 31), ([0], 13, 30, 31), ([0], 14, 30, 31), ([0], 15, 30, 31), ([0], 16, 30, 31), ([0], 17, 30, 31), ([0], 18, 30, 31), ([0], 19, 30, 31), ([0], 20, 30, 31), ([0], 21, 30, 31), ([0], 22, 30, 31), ([0], 23, 30, 31), ([0], 24, 30, 31), ([0], 25, 30, 31), ([0], 26, 30, 31), ([0], 27, 30, 31), ([0], 28, 30, 31), ([0], 29, 30, 31), ([0], 30, 30, 31), ([1], 2, 30, 31), ([1], 3, 30, 31), ([1], 4, 30, 31), ([1], 5, 30, 31), ([1], 6, 30, 31), ([1], 7, 30, 31), ([1], 8, 30, 31), ([1], 9, 30, 31), ([1], 10, 30, 31), ([1], 11, 30, 31), ([1], 12, 30, 31), ([1], 13, 30, 31), ([1], 14, 30, 31), ([1], 15, 30, 31), ([1], 16, 30, 31), ([1], 17, 30, 31), ([1], 18, 30, 31), ([1], 19, 30, 31), ([1], 20, 30, 31), ([1], 21, 30, 31), ([1], 22, 30, 31), ([1], 23, 30, 31), ([1], 24, 30, 31), ([1], 25,

[{(2, 0): 1, (3, 0): 14, (4, 0): 66, (5, 0): 104, (6, 0): 24}, {(2, 0): 1, (3, 0): 14, (4, 0): 66, (5, 0): 104, (6, 0): 24}, {(2, 0): 1, (3, 0): 20, (4, 0): 135, (5, 0): 404, (6, 0): 550, (7, 0): 288, (8, 0): 36}, {(2, 0): 1, (3, 0): 14, (4, 0): 66, (5, 0): 104, (6, 0): 24}, {(2, 0): 1, (3, 0): 19, (4, 0): 123, (5, 0): 350, (6, 0): 443, (7, 0): 209, (8, 0): 24}, {(2, 0): 1, (3, 0): 18, (4, 0): 111, (5, 0): 296, (6, 0): 336, (7, 0): 130, (8, 0): 12}, {(2, 0): 1, (3, 0): 20, (4, 0): 129, (5, 0): 354, (6, 0): 393, (7, 0): 96}, {(2, 0): 1, (3, 0): 14, (4, 0): 66, (5, 0): 104, (6, 0): 24}, {(2, 0): 1, (3, 0): 17, (4, 0): 102, (5, 0): 270, (6, 0): 318, (7, 0): 150, (8, 0): 18}, {(2, 0): 1, (3, 0): 16, (4, 0): 90, (5, 0): 216, (6, 0): 211, (7, 0): 71, (8, 0): 6}, {(2, 0): 1, (3, 0): 17, (4, 0): 97, (5, 0): 234, (6, 0): 217, (7, 0): 40}, {(2, 0): 1, (3, 0): 14, (4, 0): 69, (5, 0): 136, (6, 0): 86, (7, 0): 12}, {(2, 0): 1, (3, 0): 15, (4, 0): 75, (5, 0): 152, (6, 0): 100, (7, 0): 8}, {(2, 0): 1

In [8]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count


n=6

#computing A334254 for n=6 by levels
if __name__ == "__main__":
    #pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
   
     
    print(list(zip(lt,lk,ln_2,lnn1)))
    
    pool = Pool(len(lt))
    
    print("lt=",len(lt))

    
    #parallel execution of ProcessRT function
    res6 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res6.get())# print the list of resulting dictionaries
    print(sum(merge(res6.get()).values()))#print the number of closure systems w.r.t. T1
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 62, 63), ([0], 2, 62, 63), ([0], 3, 62, 63), ([0], 4, 62, 63), ([0], 5, 62, 63), ([0], 6, 62, 63), ([0], 7, 62, 63), ([0], 8, 62, 63), ([0], 9, 62, 63), ([0], 10, 62, 63), ([0], 11, 62, 63), ([0], 12, 62, 63), ([0], 13, 62, 63), ([0], 14, 62, 63), ([0], 15, 62, 63), ([0], 16, 62, 63), ([0], 17, 62, 63), ([0], 18, 62, 63), ([0], 19, 62, 63), ([0], 20, 62, 63), ([0], 21, 62, 63), ([0], 22, 62, 63), ([0], 23, 62, 63), ([0], 24, 62, 63), ([0], 25, 62, 63), ([0], 26, 62, 63), ([0], 27, 62, 63), ([0], 28, 62, 63), ([0], 29, 62, 63), ([0], 30, 62, 63), ([0], 31, 62, 63), ([0], 32, 62, 63), ([0], 33, 62, 63), ([0], 34, 62, 63), ([0], 35, 62, 63), ([0], 36, 62, 63), ([0], 37, 62, 63), ([0], 38, 62, 63), ([0], 39, 62, 63), ([0], 40, 62, 63), ([0], 41, 62, 63), ([0], 42, 62, 63), ([0], 43, 62, 63), ([0], 44, 62, 63), ([0], 45, 62, 63), ([0], 46, 62, 63), ([0], 47, 62, 63), ([0], 48, 62, 63), ([0], 49, 62, 63), ([0], 50, 62, 63), ([0], 51, 62, 63), ([0], 52, 62, 63), ([0], 53, 62, 63), (

lt= 1953
[{(2, 0): 1, (3, 0): 30, (4, 0): 345, (5, 0): 1720, (6, 0): 3815, (7, 0): 3520, (8, 0): 1255, (9, 0): 120}, {(2, 0): 1, (3, 0): 30, (4, 0): 345, (5, 0): 1720, (6, 0): 3815, (7, 0): 3520, (8, 0): 1255, (9, 0): 120}, {(2, 0): 1, (3, 0): 44, (4, 0): 696, (5, 0): 5578, (6, 0): 25280, (7, 0): 66402, (8, 0): 96610, (9, 0): 70154, (10, 0): 23752, (11, 0): 3736, (12, 0): 192}, {(2, 0): 1, (3, 0): 30, (4, 0): 345, (5, 0): 1720, (6, 0): 3815, (7, 0): 3520, (8, 0): 1255, (9, 0): 120}, {(2, 0): 1, (3, 0): 43, (4, 0): 668, (5, 0): 5250, (6, 0): 23252, (7, 0): 59444, (8, 0): 83663, (9, 0): 58011, (10, 0): 18361, (11, 0): 2659, (12, 0): 120}, {(2, 0): 1, (3, 0): 42, (4, 0): 640, (5, 0): 4922, (6, 0): 21224, (7, 0): 52486, (8, 0): 70716, (9, 0): 45868, (10, 0): 12970, (11, 0): 1582, (12, 0): 48}, {(2, 0): 1, (3, 0): 48, (4, 0): 810, (5, 0): 6720, (6, 0): 30759, (7, 0): 79215, (8, 0): 107304, (9, 0): 63786, (10, 0): 13797, (11, 0): 1170}, {(2, 0): 1, (3, 0): 30, (4, 0): 345, (5, 0): 1720, (6, 

CPU times: user 6.14 s, sys: 6.41 s, total: 12.6 s
Wall time: 1d 9h 4min


In [12]:
print(res6.get())

[{(2, 0): 1, (3, 0): 30, (4, 0): 345, (5, 0): 1720, (6, 0): 3815, (7, 0): 3520, (8, 0): 1255, (9, 0): 120}, {(2, 0): 1, (3, 0): 30, (4, 0): 345, (5, 0): 1720, (6, 0): 3815, (7, 0): 3520, (8, 0): 1255, (9, 0): 120}, {(2, 0): 1, (3, 0): 44, (4, 0): 696, (5, 0): 5578, (6, 0): 25280, (7, 0): 66402, (8, 0): 96610, (9, 0): 70154, (10, 0): 23752, (11, 0): 3736, (12, 0): 192}, {(2, 0): 1, (3, 0): 30, (4, 0): 345, (5, 0): 1720, (6, 0): 3815, (7, 0): 3520, (8, 0): 1255, (9, 0): 120}, {(2, 0): 1, (3, 0): 43, (4, 0): 668, (5, 0): 5250, (6, 0): 23252, (7, 0): 59444, (8, 0): 83663, (9, 0): 58011, (10, 0): 18361, (11, 0): 2659, (12, 0): 120}, {(2, 0): 1, (3, 0): 42, (4, 0): 640, (5, 0): 4922, (6, 0): 21224, (7, 0): 52486, (8, 0): 70716, (9, 0): 45868, (10, 0): 12970, (11, 0): 1582, (12, 0): 48}, {(2, 0): 1, (3, 0): 48, (4, 0): 810, (5, 0): 6720, (6, 0): 30759, (7, 0): 79215, (8, 0): 107304, (9, 0): 63786, (10, 0): 13797, (11, 0): 1170}, {(2, 0): 1, (3, 0): 30, (4, 0): 345, (5, 0): 1720, (6, 0): 3815,

In [10]:
merge(res6.get())

{(2, 0): 62,
 (3, 0): 1590,
 (4, 0): 19480,
 (5, 0): 125805,
 (6, 0): 460146,
 (7, 0): 976455,
 (8, 0): 1145880,
 (9, 0): 653220,
 (10, 0): 166380,
 (11, 0): 19770,
 (12, 0): 720,
 (2, 1): 31,
 (3, 1): 1395,
 (4, 1): 27950,
 (5, 1): 320915,
 (6, 1): 2319233,
 (7, 1): 10818181,
 (8, 1): 32286640,
 (9, 1): 60040620,
 (10, 1): 67468760,
 (11, 1): 45275855,
 (12, 1): 18291505,
 (13, 1): 4346480,
 (14, 1): 556650,
 (15, 1): 32855,
 (16, 1): 600,
 (3, 2): 1335,
 (4, 2): 26360,
 (5, 2): 297080,
 (6, 2): 2100531,
 (7, 2): 9559451,
 (8, 2): 27739807,
 (9, 2): 49856829,
 (10, 2): 53577959,
 (11, 2): 33843175,
 (12, 2): 12621962,
 (13, 2): 2707472,
 (14, 2): 298202,
 (15, 2): 13339,
 (16, 2): 120,
 (2, 2): 30,
 (2, 3): 15,
 (3, 3): 976,
 (4, 3): 23197,
 (5, 3): 294875,
 (6, 3): 2271521,
 (7, 3): 11058108,
 (8, 3): 34031407,
 (9, 3): 64449505,
 (10, 3): 72042940,
 (11, 3): 45964576,
 (12, 3): 16663660,
 (13, 3): 3370718,
 (14, 3): 357544,
 (15, 3): 19200,
 (16, 3): 576,
 (4, 4): 23304,
 (5, 4): 25